In [412]:
import numpy as np
import pandas as pd
from numpy.random import randn
np.random.seed(101)
DATA_DIR = '/Users/nathanzhao/Desktop/Python_quant_finance' + \
        '/Python-for-Finance-Repo-master-master' + \
        '/03-General Pandas' 

# Catalog:
* ### Series
* ### DataFrame
* ### Missing Data
* ### Groupby, then aggregate
* ### Concatenate, Merge by key (of columns), Join (merge with index as key)
* ### Operations: Unique, "Apply" method, Column/index names, Sort, Is null, Pivot tables
* ### Data Input & Output


================================================
================================================

## Series

### Create Pandas Series,
* arguments: data & index 

In [17]:
ls = np.array([10,20,30])
labels = ['a','b','c']
ser1 = pd.Series(data = ls, index = labels)

In [18]:
ser1['a']
ser1[0]

10

In [19]:
ser1 = pd.Series([1,2,3], index = ['a','b','c'])
ser2 = pd.Series([1,2,3], index = ['a','b','d'])
ser1 + ser2

a    2.0
b    4.0
c    NaN
d    NaN
dtype: float64

-------------
-------------

# DataFrame
* Create DataFrame
* Create new column
* Drop a column/row
* Selection of columns: ["..."]
* Selection of rows: loc[...], iloc[...], [T/F]
* Selection of rows & columns: loc[..., ...], iloc[..., ...], [T/F][...]
* Index: reset index, set index (to column)
* Index Hierarchy
* Selection from hierarchical dataframe: 【loc[...].loc[...]】, 【iloc[...]】,【xs(..., level = )】"Cross Section"

### Create DataFrame, 
* arguments: data, index, columns
* each column is just a pd.Series

In [173]:
df = pd.DataFrame(data = randn(5,4), index = ['a','b','c','d','e'], columns = ['w','x','y','z'])

### Create new column
* from combination of existing column

In [166]:
df['new'] = df['x'] + df['w']
df.shape

(5, 5)

### Drop a column/row,
*  arguments: labels, axis, inplace

In [167]:
df.drop(labels = 'new', axis = 1, inplace = True)
df.drop(labels = 'e', axis = 0, inplace = False)
df.shape

(5, 4)

### Selection of columns: ["..."]

In [168]:
df['w']
type(df['w'])

pandas.core.series.Series

In [169]:
# BEST
df[['w','y']]
type(df[['w','y']])

pandas.core.frame.DataFrame

### Selection of rows: loc[...], iloc[...], [T/F]

In [170]:
df.loc['a']
df.iloc[1]
type(df.loc['a'])

pandas.core.series.Series

In [171]:
# BEST
df.loc[['a','b']]
df.iloc[[0,1]]

df[[True,False,True,False,False]]
df.loc[[True,False,True,False,False]]
df.iloc[[True,False,True,False,False]]

# BEST
df[ (df['w'] + df['y'])>0 ]
df[ (df['w']>0) | (df['y']>0)]
df[ (df['w']>0) & (df['y']>0)]

df.loc[      (df['w']>0) & (df['y']>0) ]
df.iloc[list((df['w']>0) & (df['y']>0))]
df.iloc[    ((df['w']>0) & (df['y']>0)).values]

type(df[ (df['w']>0) | (df['y']>0)])

pandas.core.frame.DataFrame

### Selection of rows & columns: loc[..., ...], iloc[..., ...], [T/F][..]

In [202]:
df.loc[['a','b'], ['w','w']]
df.iloc[[0,1], [2,3]]

df.loc[[True,False,True,False,False], ['w','w']]
df.iloc[[True,False,True,False,False],[2,3]]

df.loc[(df['w']>0) | (df['y']>0), ['w']]
df.iloc[((df['w']>0) | (df['y']>0)).values, [1,2]]

# BEST
df[(df['w']>0) | (df['y']>0)]   ['w']
df.loc[(df['w']>0) | (df['y']>0)]   [['w','y']]
df.iloc[((df['w']>0) | (df['y']>0)).values]   [['w','y']]

type(df.iloc[((df['w']>0) | (df['y']>0)).values, [1,2]])

pandas.core.frame.DataFrame

#### Conditional Selection (useless demo)
* Get "NaN" when Conditional select on the entire df
* No "NaN" when selecting rows: see up there

In [186]:
booldf = df > 0
booldf_1 = df < -0.5
booldf_2 = booldf | booldf_1
df[booldf_2]

,w,x,y,z
a,NaN,1.062710,1.752014,0.695547
b,0.153661,0.167638,-0.765930,0.962299
c,0.902826,-0.537909,-1.549671,0.435253
d,1.259904,NaN,0.266207,0.412580
e,0.988773,0.513833,-0.928205,0.846904


### Index: reset index, set index (to column)
* argument: inplace

In [207]:
df.reset_index(inplace = False).shape

(5, 5)

In [215]:
newind = 'CA NY WY OR CO'.split()
# Create a new column with this list of new index
df['states'] = newind
# Set index to column: This will overwrite the old index, need to manually retain it
df.set_index(keys = 'states', inplace = False).shape

(5, 4)

### Index Hierarchy

In [219]:
outside = 'g1 g1 g1 g2 g2 g2'.split()
inside = [1,2,3,1,2,3]
hier_index = list(zip(outside, inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)

In [225]:
df = pd.DataFrame(data = randn(6,2), index = hier_index, columns = ['a','b'])
df.index.names = ['groups', 'num']
df

a         b
g1 1 -0.854860  1.602816
   2  0.185479 -0.994018
   3 -0.131906 -1.335092
g2 1 -0.089298  0.698566
   2 -1.409126  0.870907
   3  1.585812  0.929658

### Selection from hierarchical dataframe: 【loc[...].loc[...]】, 【iloc[...]】,【xs(..., level = )】"Cross Section"
* xs(..., level = )只能用在 MultiIndex 上
* xs(...) 可以用在 MultiIndex 或 Index 上，（replace loc？）

In [248]:
df.loc['g1']
df.loc['g1'].loc[1]
df.loc['g1'].loc[1]['b']

df.xs(1, level = 'num')
df.xs(1, level = 'num').loc['g1']
df.xs(1, level = 'num') .xs('g1')
df.xs(1, level = 'num') .xs('g1') ['b']

1.6028155369261388

--------------
-------------

# Missing Data
* Drop missing row/column: dropna()
* Fill in missing values: fillna()

### Drop missing row/column: dropna()
* arguments: axis, inplace, thresh (require that many non-nan values to not get dropped)

In [249]:
dic = {'a': [1,2,np.nan], 'b': [5,np.nan, np.nan], 'c':[1,2,3]}
df = pd.DataFrame(dic)

In [262]:
df.dropna(axis = 1, inplace = False, thresh = 2).shape

(3, 2)

### Fill in missing values: fillna()
* arguments: value, axis
* pre-decided value, mean of column

In [266]:
df.fillna(value = 'FILL')

# Fill in the mean of the column
df['a'].fillna(value = df['a'].mean())    .shape

(3,)

-------------
-------------

# Groupby, then aggregate
* by column name
* useful AGGREGATE: 
    * mean() (sum, std..) only numerical column
    * count(), max()
    * describe()

In [276]:
data = {'company' : ['GO','GO','MS', 'MS','FB','FB'], 
        'person' : ['a','b','c','d','e','f'],
        'sales' : [200, 120, 340, 124, 243, 350],
        'salary': [100, 200, 300, 200, 100, 300]
       }
df =  pd.DataFrame(data = data)

In [294]:
# Generate groupby object
byComp = df.groupby('company')
# Call AGGREGATE function, e.g. mean() => automatically ignore non-numerical
byComp.mean()

df.groupby('company')  .mean().loc['FB']['salary']
df.groupby('company')['salary']  .mean().loc['FB']

df.groupby('company')  .count()

df.groupby('company')  .describe().loc['FB'].shape

(16,)

-------------
-------------

# Concatenate, Merge by key, Join (merge with index as key)
* concat: objs, axis
* merge: left, right, how (inner, outer, left, right), on ([keys...])
* join: other, how

In [297]:
df1 = pd.DataFrame(data = randn(4,4), index = [0,1,2,3], columns = ['a','b','c','d',])
df2 = pd.DataFrame(data = randn(4,4), index = [4,5,6,7], columns = ['a','b','c','d',])
df3 = pd.DataFrame(data = randn(4,4), index = [8,9,10,11], columns = ['a','b','c','d',])

In [318]:
(pd.concat(objs = [df1,df2,df3], axis = 0).shape,  \
pd.concat(objs = [df1,df2,df3], axis = 1).shape)

((12, 4), (12, 12))

In [335]:
left = pd.DataFrame({'key1': ['K0', 'K1', 'K0', 'K1', ],
                     'key2': ['K0', 'K0', 'K1', 'K1', ],
                     'A': list(np.arange(0,4,1)),
                     'B': list(np.arange(0,-4,-1))})
right = pd.DataFrame({'key1': ['K1', 'K0', 'K3', 'K2', ],
                      'key2': ['K1', 'K0', 'K3', 'K2', ],
                     'C': list(np.arange(0,4,1)),
                     'D': list(np.arange(0,-4,-1))})

In [336]:
print(
pd.merge(left = left, right = right, how = 'inner', on = ['key1','key2']).shape,  \
pd.merge(left = left, right = right, how = 'outer', on = ['key1','key2']).shape,  \
pd.merge(left = left, right = right, how = 'left', on = ['key1','key2']).shape,  \
pd.merge(left = left, right = right, how = 'right', on = ['key1','key2']).shape,
)

(2, 6) (6, 6) (4, 6) (4, 6)


In [337]:
left = pd.DataFrame({'A': list(randn(3)),
                    'B': list(randn(3))},
                   index = ['a','b','c'])
right = pd.DataFrame({'C': list(randn(3)),
                    'D': list(randn(3))},
                   index = ['a','c','d'])

In [339]:
left.join(other = right, how = 'left').shape

(3, 4)

-------------
-------------

# Operations: 
* Unique values in dataframe (/column)
* "Apply" method
* Get column/index names
* Sort dataframe
* whether is null
* Pivot tables

### Find (# of) unique values in a dataframe (/column)
* Value counts in column

In [347]:
df = pd.DataFrame({'col1': range(1,5),
                   'col2': [444,555,666,444],
                   'col3': ['ae','bcd','cdf','d']})

In [348]:
df['col2'].unique()
df['col2'].nunique()

df['col2'].value_counts()

444    2
555    1
666    1
Name: col2, dtype: int64

### "Apply" method

In [357]:
def times2(x):
    return x*2
df['col2'].apply(times2)

# Also default functions too
df['col3'].apply(len)

# BEST: lambda function
df['col2'] = df['col2'].apply(lambda x: x*2)  

### Get column/index names

In [359]:
df.columns
df.index

RangeIndex(start=0, stop=4, step=1)

### Sorting dataframe
* by a column (index stays attached)

In [362]:
df.sort_values(by = 'col2', axis = 0)     .shape

(4, 3)

### Whether a value of null or not

In [364]:
df.isnull()     .shape

(4, 3)

### Pivot table:
*  arguments: values, index, columns

In [365]:
data = {'A':['foo','foo','foo','bar','bar','bar'],
     'B':['one','one','two','two','one','one'],
       'C':['x','y','x','y','x','y'],
       'D':[1,3,2,5,4,1]}
df = pd.DataFrame(data)

In [374]:
df.pivot_table(values='D', index = ['A','B'], columns = ['C'])
df.pivot_table(values='D', index = ['A'], columns = ['B','C'])
df.pivot_table(values='D', index = ['A','B'], columns = ['B','C'])

B        one       two     
C          x    y    x    y
A   B                      
bar one  4.0  1.0  NaN  NaN
    two  NaN  NaN  NaN  5.0
foo one  1.0  3.0  NaN  NaN
    two  NaN  NaN  2.0  NaN

-------------
-------------

# Data Input & Output
* CSV
* Excel
* HTML
* SQL

In [376]:
DATA_DIR = '/Users/nathanzhao/Desktop/Python_quant_finance' + \
        '/Python-for-Finance-Repo-master-master' + \
        '/03-General Pandas' 


### CSV: read_csv, to_csv (from df)
* to_csv: index T/F

In [384]:
df = \
pd.read_csv(DATA_DIR+'/example')    

In [386]:
df.to_csv(DATA_DIR+ '/output', index = False)

### Excel: read_excel, to_excel
* argument: sheet_name, index_col
* no graph or formula allowed; 
* may need xlrd library

In [395]:
df =pd.read_excel(DATA_DIR+ '/Excel_Sample.xlsx', sheet_name = 'Sheet1', index_col = 0)

In [397]:
df.to_excel(DATA_DIR+ '/Excel_output.xlsx', sheet_name = 'Sheet1')

### HTML:
* loop through the list from read_html, until see the right thing


In [399]:
data = pd.read_html('https://www.fdic.gov/bank/individual/failed/banklist.html')
type(data)

list

In [404]:
df = data[0]
type(df)

pandas.core.frame.DataFrame

### SQL
* to_sql: name (of table), con (connection)
* read_sql: sql (of table), con (connection)

In [407]:
from sqlalchemy import create_engine
engine  = create_engine('sqlite:///:memory:')

In [409]:
df.to_sql(name = 'my_table', con = engine)

In [410]:
sql_df = pd.read_sql(sql = 'my_table', con = engine)